# imports

In [ ]:
import os
import boto3

In [ ]:
def get_boto_client(service, **kwargs):
    return boto3.client(
        service,
        aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
        aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
        **kwargs
    )

client = get_boto_client('iam')

### validate we can make an API call

In [ ]:
client.get_role(RoleName='my-lambda-role')

### create-change-set

In [ ]:
cfn_client = get_boto_client('cloudformation')

### initial template we are importing has drift

In [ ]:
with open('../templates/001.yaml') as f:
    template_body = f.read()

In [ ]:
resp = cfn_client.create_change_set(
    StackName='MyImportStack',
    TemplateBody=template_body,
    ChangeSetType='IMPORT',
    ChangeSetName="MyImportStack001",
    Capabilities=['CAPABILITY_NAMED_IAM'],
    ResourcesToImport=[
   {
      "ResourceType":"AWS::IAM::Role",
      "LogicalResourceId":"MyRole",
      "ResourceIdentifier":{
         "RoleName":"my-lambda-role"
      }
   }]
)

resp

In [ ]:
resp2 = cfn_client.describe_change_set(
    StackName='MyImportStack',
    ChangeSetName="MyImportStack001",
)

resp2

### cfn stack will be in REVIEW_IN_PROGRESS until we make this call

In [ ]:
resp3 = cfn_client.execute_change_set(
    StackName='MyImportStack',
    ChangeSetName="MyImportStack001",
)

resp3

### we now see Drift status is DRIFTED but Role is unchanged in AWS, so that's ok

In [ ]:
resp4 = cfn_client.detect_stack_drift(
    StackName='MyImportStack'
)
resp4

In [ ]:
resp5 = cfn_client.describe_stack_drift_detection_status(
    StackDriftDetectionId=resp4['StackDriftDetectionId']
)
resp5

In [ ]:
resp6 = cfn_client.describe_stack_resource_drifts(
    StackName='MyImportStack'
)
resp6

## update stack to match (no drift)

In [ ]:
with open('../templates/002.yaml') as f:
    template_body2 = f.read()

In [ ]:
response1 = cfn_client.update_stack(
    StackName='MyImportStack',
    TemplateBody=template_body2,
    Capabilities=['CAPABILITY_NAMED_IAM'],
)
response1

In [ ]:
response2 = cfn_client.detect_stack_drift(
    StackName='MyImportStack'
)
response2

### we now see Drift status is IN_SYNC and CFN stack matches the Role

In [ ]:
response3 = cfn_client.describe_stack_drift_detection_status(
    StackDriftDetectionId=response2['StackDriftDetectionId']
)
response3

### Update the role via the CFN stack

In [ ]:
with open('../templates/001.yaml') as f:
    template_body1 = f.read()

In [ ]:
response10 = cfn_client.update_stack(
    StackName='MyImportStack',
    TemplateBody=template_body1,
    Capabilities=['CAPABILITY_NAMED_IAM'],
)
response10